In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vimle\\Desktop\\vae\\Text-summarizer1\\Text-Summarizer-pp\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vimle\\Desktop\\vae\\Text-summarizer1\\Text-Summarizer-pp'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [6]:
from src.TextSummarizer.constants.__init__ import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_Ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.TextSummarizer.logging import logger
from src.TextSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with info : \n {headers}")
        else:
            logger.info(f"File already exists. {get_size(Path(self.config.local_data_fike))} Skipping download")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-03 00:39:44,972: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2024-11-03 00:39:44,973: INFO: common: yaml file: params.yaml loaded succesfully]
[2024-11-03 00:39:44,974: INFO: common: Directory artifacts created succesfully]
[2024-11-03 00:39:44,974: INFO: common: Directory artifacts/data_Ingestion created succesfully]
[2024-11-03 00:39:48,279: INFO: 4032732299: artifacts/data_Ingestion/data.zip download with info : 
 Connection: close
Content-Length: 24408865
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "1373c6065e27716837297a80974fad9d9b7ecb2487870f42a93ed9683808345d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4CA1:33EBF:57F777:70974F:6726786E
Accept-Ranges: bytes
Date: Sat, 02 Nov 2024 19:09:43 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10249-MAA
X-Cac